# First webscraping task

In [1]:
# Load BeautifulSoup, and pandas to store in a dataframe
from bs4 import BeautifulSoup
import pandas as pd

In [6]:
example_html = """
<!DOCTYPE html>
<html>
<head>
<title>Sample web page</title>
</head>
<body>
<h1>h1 Header #1</h1>
<p>This is a paragraph tag</p>
<h2>h2 Sub-header</h2>
<p>A new paragraph, now in the <b>sub-header</b></p>
<h1>h1 Header #2</h1>
<p>This other paragraph has two  hyperlinks, one to <a href="https://carpentries.org/">The Carpentries homepage</a>, and another to the <a href="https://carpentries.org/past_workshops/">past workshops</a>.</p>
</body>
</html>
"""

In [10]:
help(BeautifulSoup)

Help on class BeautifulSoup in module bs4:

class BeautifulSoup(bs4.element.Tag)
 |  BeautifulSoup(markup='', features=None, builder=None, parse_only=None, from_encoding=None, exclude_encodings=None, element_classes=None, **kwargs)
 |  
 |  A data structure representing a parsed HTML or XML document.
 |  
 |  Most of the methods you'll call on a BeautifulSoup object are inherited from
 |  PageElement or Tag.
 |  
 |  Internally, this class defines the basic interface called by the
 |  tree builders when converting an HTML/XML document into a data
 |  structure. The interface abstracts away the differences between
 |  parsers. To write a new tree builder, you'll need to understand
 |  these methods as a whole.
 |  
 |  These methods will be called by the BeautifulSoup constructor:
 |    * reset()
 |    * feed(markup)
 |  
 |  The tree builder may call these methods from its feed() implementation:
 |    * handle_starttag(name, attrs) # See note about return value
 |    * handle_endtag(na

Create soup object, and see tree structure

In [11]:
soup = BeautifulSoup(example_html, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   Sample web page
  </title>
 </head>
 <body>
  <h1>
   h1 Header #1
  </h1>
  <p>
   This is a paragraph tag
  </p>
  <h2>
   h2 Sub-header
  </h2>
  <p>
   A new paragraph, now in the
   <b>
    sub-header
   </b>
  </p>
  <h1>
   h1 Header #2
  </h1>
  <p>
   This other paragraph has two  hyperlinks, one to
   <a href="https://carpentries.org/">
    The Carpentries homepage
   </a>
   , and another to the
   <a href="https://carpentries.org/past_workshops/">
    past workshops
   </a>
   .
  </p>
 </body>
</html>



In [15]:
type(soup)

bs4.BeautifulSoup

In [22]:
# Accessing different parts of the website
print("1.", soup.find('title'))
print("2.", soup.find('title').get_text())
print("3.", soup.find('h1').get_text())
print("4.", soup.find_all('h1'))
print("5.", soup.find_all('a'))
print("6.", soup.get_text(separator='\n', strip=True))

1. <title>Sample web page</title>
2. Sample web page
3. h1 Header #1
4. [<h1>h1 Header #1</h1>, <h1>h1 Header #2</h1>]
5. [<a href="https://carpentries.org/">The Carpentries homepage</a>, <a href="https://carpentries.org/past_workshops/">past workshops</a>]
6. Sample web page  h1 Header #1  This is a paragraph tag  h2 Sub-header  A new paragraph, now in the  sub-header  h1 Header #2  This other paragraph has two  hyperlinks, one to  The Carpentries homepage  , and another to the  past workshops  .


In [23]:
# How many elements with the 'a' tag did we find?
# find_all returns a list with all the tags that we specified. There could be 0, 1 or multiple items in the list
links = soup.find_all('a')
print("Number of hyperlinks found: ", len(links))
print(links)

Number of hyperlinks found:  2
[<a href="https://carpentries.org/">The Carpentries homepage</a>, <a href="https://carpentries.org/past_workshops/">past workshops</a>]


In [65]:
# Now create a dataframe with all hyperlinks and the text each is referencing
for item in links:
    print(item)

<a href="https://carpentries.org/">The Carpentries homepage</a>
<a href="https://carpentries.org/past_workshops/">past workshops</a>


In [42]:
type(soup.find_all('a')[0].string)

bs4.element.NavigableString

In [25]:
# We can access the attribute like treating them as a dictionary
# And we can access the text using .text
for item in links:
    print(item['href'])

https://carpentries.org/
https://carpentries.org/past_workshops/


Maybe a refresher on dictionaries?

In [45]:
# Storing all as a list of dictionaries to create a table

# Creating empty dictionary
list_of_dicts = []
for item in soup.find_all('a'):
    dict_a = {}
    dict_a['element'] = find_all('a')
    dict_a['url'] = item['href']
    dict_a['text'] = item.text
    list_of_dicts.append(dict_a)

In [31]:
first_link = {
   'element': str(soup.find('a')),
   'url': soup.find('a')['href'],
   'text': soup.find('a').get_text()
}

print(first_link)

{'element': '<a href="https://carpentries.org/">The Carpentries homepage</a>', 'url': 'https://carpentries.org/', 'text': 'The Carpentries homepage'}


In [37]:
links = soup.find_all('a')
list_of_dicts = []
for item in links:
    dict_a = {}
    dict_a['element'] = str(item)
    dict_a['url'] = item['href']
    dict_a['text'] = item.get_text()
    list_of_dicts.append(dict_a)

links_df = pd.DataFrame(list_of_dicts)
print(links_df)

                                             element  \
0  <a href="https://carpentries.org/">The Carpent...   
1  <a href="https://carpentries.org/past_workshop...   

                                       url                      text  
0                 https://carpentries.org/  The Carpentries homepage  
1  https://carpentries.org/past_workshops/            past workshops  


In [40]:
type(list_of_dicts[0]['url'])

str

## A more real example, actually getting information from a website

In [42]:
# Check the website, see that we want to obtain the data from the upcoming workshops table


In [101]:
# We'll use the requests package to get the HTML from a website, the Carpentries website
import requests
import re
url = 'https://carpentries.org/'
req = requests.get(url).text
cleaned_req = re.sub(r'\s*\n\s*', '', req).strip()
print(cleaned_req[0:1000])

<!doctype html><html class="no-js" lang="en"><head><meta charset="utf-8"><meta name="viewport" content="width=device-width, initial-scale=1.0"><title>The Carpentries</title><link rel="stylesheet" type="text/css" href="https://carpentries.org/assets/css/styles_feeling_responsive.css"><script src="https://carpentries.org/assets/js/modernizr.min.js"></script><!-- matomo --><script src="https://carpentries.org/assets/js/matomo-analytics.js"></script><link href="https://fonts.googleapis.com/css?family=Lato:400,400i,700,700i|Roboto:400,400i,700,700i&display=swap" rel="stylesheet"><!-- Search Engine Optimization --><meta name="description" content="The Carpentries is a fiscally sponsored project of Community Initiatives, a registered 501(c)3 non-profit organisation based in California, USA. We are a global community teaching foundational computational and data science skills to researchers in academia, industry and government."><link rel="canonical" href="https://carpentries.org/index.html"><

- Go to The Carpentries website, and compare the previous result to the 'View source' in Chrome

In 'View source', try to find 'Upcoming Carpentries Workshops'.
Or alternatively, select an element of the table and click 'Inspect'

In [102]:
soup = BeautifulSoup(cleaned_req, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="no-js" lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <title>
   The Carpentries
  </title>
  <link href="https://carpentries.org/assets/css/styles_feeling_responsive.css" rel="stylesheet" type="text/css"/>
  <script src="https://carpentries.org/assets/js/modernizr.min.js">
  </script>
  <!-- matomo -->
  <script src="https://carpentries.org/assets/js/matomo-analytics.js">
  </script>
  <link href="https://fonts.googleapis.com/css?family=Lato:400,400i,700,700i|Roboto:400,400i,700,700i&amp;display=swap" rel="stylesheet"/>
  <!-- Search Engine Optimization -->
  <meta content="The Carpentries is a fiscally sponsored project of Community Initiatives, a registered 501(c)3 non-profit organisation based in California, USA. We are a global community teaching foundational computational and data science skills to researchers in academia, industry and government." name="description"/>
  <link h

In [103]:
soup = BeautifulSoup(cleaned_req, 'html.parser')
tables_by_tag = soup.find_all('table')
print("Number of table elements found: ", len(tables_by_tag))
print("Printing only the first 1000 characters of the table element: \n", str(tables_by_tag[0])[0:1000])

Number of table elements found:  1
Printing only the first 1000 characters of the table element: 
 <table class="table table-striped" style="width: 100%;"><tr><td><img alt="swc logo" class="flags" height="24" src="https://carpentries.org/assets/img/logos/swc.svg" title="swc workshop" width="24"/></td><td><img alt="mx" class="flags" src="https://carpentries.org/assets/img/flags/24/mx.png" title="MX"><img alt="globe image" class="flags" src="https://carpentries.org/assets/img/flags/24/w3.png" title="Online"><a href="https://galn3x.github.io/-2024-10-28-Metagenomics-online/">Nodo Nacional de BioinformÃ¡tica UNAM</a><br><b>Instructors:</b> CÃ©sar Aguilar, Diana Oaxaca, Nelly Selem-Mojica<br><b>Helpers:</b> Andreas Chavez, JosÃ© Manuel Villalobos Escobedo, Aaron Espinosa Jaime, AndrÃ©s Arredondo, Mirna VÃ¡zquez Rosas-Landa, David Alberto GarcÃ­a-Estrada</br></br></img></img></td><td>Oct 28 - Oct 31, 2024</td></tr><tr><td><img alt="dc logo" class="flags" height="24" src="https://carpentries.

In [104]:
tables_by_class = soup.find_all(class_="table table-striped")
print("Number of table elements found: ", len(tables_by_class))
print(str(tables_by_class[0])[0:1000])

Number of table elements found:  1
<table class="table table-striped" style="width: 100%;"><tr><td><img alt="swc logo" class="flags" height="24" src="https://carpentries.org/assets/img/logos/swc.svg" title="swc workshop" width="24"/></td><td><img alt="mx" class="flags" src="https://carpentries.org/assets/img/flags/24/mx.png" title="MX"><img alt="globe image" class="flags" src="https://carpentries.org/assets/img/flags/24/w3.png" title="Online"><a href="https://galn3x.github.io/-2024-10-28-Metagenomics-online/">Nodo Nacional de BioinformÃ¡tica UNAM</a><br><b>Instructors:</b> CÃ©sar Aguilar, Diana Oaxaca, Nelly Selem-Mojica<br><b>Helpers:</b> Andreas Chavez, JosÃ© Manuel Villalobos Escobedo, Aaron Espinosa Jaime, AndrÃ©s Arredondo, Mirna VÃ¡zquez Rosas-Landa, David Alberto GarcÃ­a-Estrada</br></br></img></img></td><td>Oct 28 - Oct 31, 2024</td></tr><tr><td><img alt="dc logo" class="flags" height="24" src="https://carpentries.org/assets/img/logos/dc.svg" title="dc workshop" width="24"/></t

In [107]:
# Access the only element, the only table
print("Before: ", type(tables_by_tag))
workshops_table = tables_by_tag[0]
print("After:", type(workshops_table))
print("Element type:", workshops_table.name)

Before:  <class 'bs4.element.ResultSet'>
After: <class 'bs4.element.Tag'>
Element type: table


In [115]:
print(workshops_table.contents[0].prettify())

<tr>
 <td>
  <img alt="swc logo" class="flags" height="24" src="https://carpentries.org/assets/img/logos/swc.svg" title="swc workshop" width="24"/>
 </td>
 <td>
  <img alt="mx" class="flags" src="https://carpentries.org/assets/img/flags/24/mx.png" title="MX">
   <img alt="globe image" class="flags" src="https://carpentries.org/assets/img/flags/24/w3.png" title="Online">
    <a href="https://galn3x.github.io/-2024-10-28-Metagenomics-online/">
     Nodo Nacional de BioinformÃ¡tica UNAM
    </a>
    <br>
     <b>
      Instructors:
     </b>
     CÃ©sar Aguilar, Diana Oaxaca, Nelly Selem-Mojica
     <br>
      <b>
       Helpers:
      </b>
      Andreas Chavez, JosÃ© Manuel Villalobos Escobedo, Aaron Espinosa Jaime, AndrÃ©s Arredondo, Mirna VÃ¡zquez Rosas-Landa, David Alberto GarcÃ­a-Estrada
     </br>
    </br>
   </img>
  </img>
 </td>
 <td>
  Oct 28 - Oct 31, 2024
 </td>
</tr>



In [110]:
print(workshops_table.prettify())

<table class="table table-striped" style="width: 100%;">
 <tr>
  <td>
   <img alt="swc logo" class="flags" height="24" src="https://carpentries.org/assets/img/logos/swc.svg" title="swc workshop" width="24"/>
  </td>
  <td>
   <img alt="mx" class="flags" src="https://carpentries.org/assets/img/flags/24/mx.png" title="MX">
    <img alt="globe image" class="flags" src="https://carpentries.org/assets/img/flags/24/w3.png" title="Online">
     <a href="https://galn3x.github.io/-2024-10-28-Metagenomics-online/">
      Nodo Nacional de BioinformÃ¡tica UNAM
     </a>
     <br>
      <b>
       Instructors:
      </b>
      CÃ©sar Aguilar, Diana Oaxaca, Nelly Selem-Mojica
      <br>
       <b>
        Helpers:
       </b>
       Andreas Chavez, JosÃ© Manuel Villalobos Escobedo, Aaron Espinosa Jaime, AndrÃ©s Arredondo, Mirna VÃ¡zquez Rosas-Landa, David Alberto GarcÃ­a-Estrada
      </br>
     </br>
    </img>
   </img>
  </td>
  <td>
   Oct 28 - Oct 31, 2024
  </td>
 </tr>
 <tr>
  <td>
   <img al

In [98]:
for item in table.children:
    print(type(item))

<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>


In [75]:
req

'<!doctype html>\n<html class="no-js" lang="en">\n<head>\n\t<meta charset="utf-8">\n\t<meta name="viewport" content="width=device-width, initial-scale=1.0">\n\t<title>The Carpentries</title>\n\n    <link rel="stylesheet" type="text/css" href="https://carpentries.org/assets/css/styles_feeling_responsive.css">\n\n  \n\n\t<script src="https://carpentries.org/assets/js/modernizr.min.js"></script>\n\n        <!-- matomo -->\n        <script src="https://carpentries.org/assets/js/matomo-analytics.js"></script>\n\n        <link href="https://fonts.googleapis.com/css?family=Lato:400,400i,700,700i|Roboto:400,400i,700,700i&display=swap" rel="stylesheet">\n\n\t<!-- Search Engine Optimization -->\n\t<meta name="description" content="The Carpentries is a fiscally sponsored project of Community Initiatives, a registered 501(c)3 non-profit organisation based in California, USA. We are a global community teaching foundational computational and data science skills to researchers in academia, industry a

In [74]:
req2

'<!doctype html>\n<html class="no-js" lang="en">\n<head>\n<meta charset="utf-8">\n<meta name="viewport" content="width=device-width, initial-scale=1.0">\n<title>The Carpentries</title>\n<link rel="stylesheet" type="text/css" href="https://carpentries.org/assets/css/styles_feeling_responsive.css">\n<script src="https://carpentries.org/assets/js/modernizr.min.js"></script>\n<!-- matomo -->\n<script src="https://carpentries.org/assets/js/matomo-analytics.js"></script>\n<link href="https://fonts.googleapis.com/css?family=Lato:400,400i,700,700i|Roboto:400,400i,700,700i&display=swap" rel="stylesheet">\n<!-- Search Engine Optimization -->\n<meta name="description" content="The Carpentries is a fiscally sponsored project of Community Initiatives, a registered 501(c)3 non-profit organisation based in California, USA. We are a global community teaching foundational computational and data science skills to researchers in academia, industry and government.">\n<link rel="canonical" href="https://ca

In [72]:
text = "\n\n  \n\nSome text\n\n   More text\n\n\n"

# Replace multiple newlines with a single newline
cleaned_text = re.sub(r'\s*\n\s*', '\n', text).strip()
cleaned_text

'Some text\nMore text'

In [44]:
for element in soup.find_all(string=True):
    element.replace_with(element.strip())

C:\Users\jmuriel\AppData\Local\Temp\ipykernel_13072\3717853505.py:1: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for element in soup.find_all(text=True):


Using the 'Inspect' feature, we can explore the structure of the table element
We could see that it has multiple child tags, called 'tr' for 'table row', and each 'tr' has a 'td' for table data

In [5]:
# Let's get all rows and count how many there are
rows = table.find_all('tr')
len(rows)

20

In [133]:
# Check first row
print(rows[0])

<tr>
<td>
<img alt="cp logo" class="flags" height="24" src="https://carpentries.org/assets/img/logos/cp.svg" title="cp workshop" width="24">
</img></td>
<td>
<img alt="us" class="flags" src="https://carpentries.org/assets/img/flags/24/us.png" title="US">
<a href="https://ucsbcarpentry.github.io/2024-10-08-ucsb/">UC Santa Barbara Library **</a>
<br>
<b>Instructors:</b> Jose Nino Muriel, Greg JanÃ©e, Julien Brun, Renata Curty
      
      
          <br>
<b>Helpers:</b> Jay Chi, Seth Erickson, Nandini Iyer, Sigrid Van Den Abbeele
      
	</br></br></img></td>
<td>
		Oct 8 - Oct 15, 2024
	</td>
</tr>


In [134]:
columns_first_row = rows[0].find_all('td')
# 3 columns:
#   0 is workshop type
#   1 is host, link, instructors and helpers
#   2 is dates

In [135]:
print("type: ", columns_first_row[0].find('img')['title']) # Getting the title attribute of the first img tag
print("location: ", columns_first_row[1].find('img')['title'])
print("link: ", columns_first_row[1].find('a')['href'])
print("host: ", columns_first_row[1].find('a').text)
print("all_text: ", columns_first_row[1].get_text(strip=True))
print("date: ", columns_first_row[2].get_text(strip=True))

type:  cp workshop
location:  US
link:  https://ucsbcarpentry.github.io/2024-10-08-ucsb/
host:  UC Santa Barbara Library **
all_text:  UC Santa Barbara Library **Instructors:Jose Nino Muriel, Greg JanÃ©e, Julien Brun, Renata CurtyHelpers:Jay Chi, Seth Erickson, Nandini Iyer, Sigrid Van Den Abbeele
date:  Oct 8 - Oct 15, 2024


In [136]:
# Doing it for all the rows in the 'table' tag
list_of_workshops = []
for row in rows:
    dict_data = {}
    columns_row = row.find_all('td')
    dict_data["type"] = columns_row[0].find('img')['title']
    dict_data["location"] =  columns_row[1].find('img')['title']
    dict_data["link"] = columns_row[1].find('a')['href']
    dict_data["host"] = columns_row[1].find('a').text
    dict_data["all_text"] = columns_row[1].get_text(strip=True)
    dict_data["date"] =  columns_row[2].get_text(strip=True)
    list_of_workshops.append(dict_data)

In [137]:
df_workshops = pd.DataFrame(list_of_workshops)
print(df_workshops.shape)
print(df_workshops.columns)

(23, 6)
Index(['type', 'location', 'link', 'host', 'all_text', 'date'], dtype='object')


## Challenge
Now do this same process for the entire list of workshops in https://carpentries.org/past_workshops/. What is the top 5 number of countries with more workshops and how many have each? 

In [139]:
url = 'https://carpentries.org/past_workshops/'
req_past = requests.get(url)
soup_past = BeautifulSoup(req_past.text, 'html.parser')

In [140]:
table = soup_past.find('table')
list_of_workshops = []
for row in table.find_all('tr'):
    dict_data = {}
    columns_row = row.find_all('td')
    dict_data["type"] = columns_row[0].find('img')['title']
    dict_data["location"] =  columns_row[1].find('img')['title']
    dict_data["link"] = columns_row[1].find('a')['href']
    dict_data["host"] = columns_row[1].find('a').text
    dict_data["all_text"] = columns_row[1].get_text(strip=True)
    dict_data["date"] =  columns_row[2].get_text(strip=True)
    list_of_workshops.append(dict_data)

In [143]:
df_past_workshops = pd.DataFrame(list_of_workshops)
print(df_past_workshops.shape)
print(df_past_workshops.columns)

(3816, 6)
Index(['type', 'location', 'link', 'host', 'all_text', 'date'], dtype='object')


In [144]:
df_past_workshops['location'].value_counts().head()

location
US    1831
GB     466
AU     333
CA     225
DE     172
Name: count, dtype: int64

# Challenge 2
You can see that there is also a little world icon for thos workshops that are online. What would your approach be to have in your dataframe a variable to distinguish if the workshops was online or not? Write code according to your approach


In [ ]:
# To do

## Creating a web of scrapers
Now, we'll go into each link and get additional information from those pages. However, when doing so, we have to be mindful about the consumption of the servers we are doing

In [150]:
first_5_websites = df_past_workshops['link'].head(5)
first_5_websites

0           https://icr-sd-cc.github.io/2024-10-09-ICR
1    https://nclrse-training.github.io/2024-10-08-NCL/
2    https://brian-maass-unmc.github.io/2024-10-07-...
3    https://tajuakins.github.io/2024-10-01-nau-onl...
4       https://librarylady1.github.io/2024-09-28-UMD/
Name: link, dtype: object

In [152]:
first_5_websites = list(first_5_websites)
first_5_websites

['https://icr-sd-cc.github.io/2024-10-09-ICR',
 'https://nclrse-training.github.io/2024-10-08-NCL/',
 'https://brian-maass-unmc.github.io/2024-10-07-UNMC/',
 'https://tajuakins.github.io/2024-10-01-nau-online/',
 'https://librarylady1.github.io/2024-09-28-UMD/']

In [171]:
first_test = first_5_websites[0]
req = requests.get(first_test)
soup = BeautifulSoup(req.text, 'html.parser')

In [172]:
soup


<!DOCTYPE html>

<html lang="en">
<head>
<meta content="2024-10-09-ICR" name="slug"/>
<meta content="2024-10-09" name="startdate"/>
<meta content="2024-10-09" name="enddate"/>
<meta content="Wed Oct 09, 2024" name="humandate"/>
<meta content="uk" name="country"/>
<meta content="ICR Sutton" name="venue"/>
<meta content="Sutton: SRD Ground Floor" name="address"/>
<meta content="51.34489470306327,-0.1888057498478897" name="latlng"/>
<meta content="en" name="language"/>
<meta content="" name="eventbrite"/>
<meta content="Rachel Alcraft" name="instructor"/>
<meta content="tbc" name="helper"/>
<meta content="rachel.alcraft@icr.ac.uk" name="contact"/>
<meta content="swc" name="carpentry"/>
<!-- meta "search-domain" used for google site search function google_search() -->
<meta name="search-domain" value="https://icr-sd-cc.github.io/2024-10-09-ICR"/>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="2024-07-26 12:43:51 +0000" http-equiv="last-modifi

In [182]:
# We looking at the HTML, we find some useful information hidden that we'd like to scrape
# We could find a tag with a specific attribute and attribute value, with the 'attrs' parameter
print(soup.find('meta', attrs ={'name': 'latlng'})['content'])
print(soup.find('meta', attrs ={'name': 'instructor'})['content'])
print(soup.find('meta', attrs ={'name': 'helper'})['content'])
print(soup.get_text(strip=True))

51.34489470306327,-0.1888057498478897


In [190]:
import time
from tqdm import tqdm

In [191]:
list_of_dicts = []
for item in tqdm(list(first_5_websites)):
    req = requests.get(item)
    soup = BeautifulSoup(req.text, 'html.parser')
    dict_new_data = {}
    dict_new_data['link'] = item
    dict_new_data['latlng'] = soup.find('meta', attrs ={'name': 'latlng'})['content']
    dict_new_data['instructor'] = soup.find('meta', attrs ={'name': 'instructor'})['content']
    dict_new_data['helper'] = soup.find('meta', attrs ={'name': 'helper'})['content']
    dict_new_data['text'] = soup.get_text(strip=True)
    list_of_dicts.append(dict_new_data)
    time.sleep(3)

100%|██████████| 5/5 [00:16<00:00,  3.27s/it]


In [192]:
new_data_df = pd.DataFrame(list_of_dicts)
new_data_df

,link,latlng,instructor,helper,text
0,https://icr-sd-cc.github.io/2024-10-09-ICR,"51.34489470306327,-0.1888057498478897",Rachel Alcraft,tbc,"ICR Sutton: Wed Oct 09, 2024Toggle navigationH..."
1,https://nclrse-training.github.io/2024-10-08-NCL/,"54.980639,-1.6143042",Dr Jannetta Steyn|Imre Draskovits,Ruxandra Neatu|Steve Debski,"Newcastle University, Henry Daysh Building 6.1..."
2,https://brian-maass-unmc.github.io/2024-10-07-...,"41.255885,-95.974518",Brian Maass|Lisa Chinn|Caughlin Bohn,Natasha Pavlovikj|Emily Nimsakont,University of Nebraska Medical Center: October...
3,https://tajuakins.github.io/2024-10-01-nau-onl...,"45,-1",Salena Torres Ashton|Aristotelis Misios|Lyrric...,helper one|helper two,"North Arizona University, Institute for Tribal..."
4,https://librarylady1.github.io/2024-09-28-UMD/,"41.633331,-71.006882",Krishna Bijjam|Zebulun Arendsee,Andy Jones|Catalina Roma,University of Massachusetts Dartmouth: Sep 28-...


If you do this for all rows without waiting, you could send 3.816 requests to the web server in a couple of seconds! This is rude, as you are taking too many resources from the server, potentially inhibiting access to other users, or maybe causing it to crash. Your IP could be identified and you could get blocked, to say the least!